In [46]:
reset -sf

In [47]:
import json
import requests
import time
from pprint import pprint
from datetime import datetime

In [48]:
in_theaters = "https://api.themoviedb.org/3/movie/now_playing?api_key=95c58b7a3d4467f15e5af2ff0c2e61b6&language=en-US&page=1"
coming_soon = "https://api.themoviedb.org/3/movie/upcoming?api_key=95c58b7a3d4467f15e5af2ff0c2e61b6&language=en-US&page=1"


In [49]:
genres_url = "https://api.themoviedb.org/3/genre/movie/list?api_key=95c58b7a3d4467f15e5af2ff0c2e61b6&language=en-US"

In [50]:
def pull_movie_data(url, dict_name, genres_url):
    
    payload1, payload2 = {}, {}
    dict_name, simply_dict = {}, {}
    
    mov_response = requests.request("GET", url, data=payload1)
    genre_response = requests.request("GET", genres_url, data=payload2 )
    ts = time.time()
    timestamp = datetime.fromtimestamp(ts).strftime("%Y-%m-%d %H:%M:%S")
    
    mov_payload = json.loads(mov_response.text)
    genre_payload = json.loads(genre_response.text)
    
    for genre in genre_payload['genres']:
        simply_dict[genre['id']] = genre['name'].encode('utf-8')
    
    for movie in mov_payload['results']:
        for key, val in simply_dict.items():
            if key == movie['genre_ids'][0]:
                genre = val
        dict_name[movie['title'].encode('utf-8')] = {'genre': genre}
    dict_name['time'] = timestamp
    return(dict_name)


In [51]:
now_playing = pull_movie_data(in_theaters, 'now_playing', genres_url)
# print('\n')
upcoming = pull_movie_data(coming_soon, 'upcoming', genres_url)

In [33]:
pprint(now_playing)
print('\n')
pprint(upcoming)

{'Arrival': {'genre': 'Drama'},
 'Fences': {'genre': 'Drama'},
 'Fifty Shades Darker': {'genre': 'Drama'},
 'Get Out': {'genre': 'Comedy'},
 'Hacksaw Ridge': {'genre': 'Drama'},
 'Hell or High Water': {'genre': 'Western'},
 'Hidden Figures': {'genre': 'History'},
 'John Wick: Chapter 2': {'genre': 'Thriller'},
 'Kong: Skull Island': {'genre': 'Science Fiction'},
 'La La Land': {'genre': 'Drama'},
 'Lion': {'genre': 'Drama'},
 'Logan': {'genre': 'Action'},
 'Manchester by the Sea': {'genre': 'Drama'},
 "Miss Peregrine's Home for Peculiar Children": {'genre': 'Drama'},
 'Moonlight': {'genre': 'Drama'},
 'Nocturnal Animals': {'genre': 'Drama'},
 'Rings': {'genre': 'Horror'},
 'Split': {'genre': 'Drama'},
 'The Lego Batman Movie': {'genre': 'Fantasy'},
 'Underworld: Blood Wars': {'genre': 'Action'},
 'time': '2017-03-07 18:50:44'}


{'A Cure for Wellness': {'genre': 'Drama'},
 'Beauty and the Beast': {'genre': 'Fantasy'},
 'Get Out': {'genre': 'Comedy'},
 'Ghost in the Shell': {'genre': 'A

----
## Utilize `IMDbpy` to retrieve necessary *MPAA ratings* for movies

In [52]:
from imdb import IMDb
conn = IMDb()

In [70]:
def collect_mpaa(conn, dict_name):
    
    for title in dict_name.keys():
        if title != 'time':
            try:
                mov_title = conn.search_movie(title)[0]
                conn.update(mov_title)
                rating = mov_title["mpaa"]
                dict_name[title]["rated"] = rating.split(' ')[1].encode('utf-8')
            except KeyError as e:
                continue
    
    return(dict_name)